In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import OneHotEncoder
import numpy as np

In [ ]:
file_path = '/content/Final Fashion Dataset.csv'
df = pd.read_csv(file_path)

In [ ]:
df_filtered = df[['gender', 'masterCategory', 'subCategory', 'articleType',
                          'baseColour', 'usage', 'ratings', 'Price (USD)', 'link']]

In [ ]:
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')  # Handle unknown categories
encoded_columns = encoder.fit_transform(df_filtered[['gender', 'baseColour', 'usage']])
encoded_df = pd.DataFrame(encoded_columns, columns=encoder.get_feature_names_out(['gender', 'baseColour', 'usage']))
df_encoded = pd.concat([df_filtered.reset_index(drop=True), encoded_df], axis=1)

In [ ]:
import colorsys

def recommend_complementary_items(input_index, num_recommendations=1):
    input_item = df_encoded.iloc[input_index]
    gender = input_item['gender']
    input_category = input_item['masterCategory']
    input_subcategory = input_item['subCategory']

    recommendation_categories = {
    # Men's Apparel - Topwear
    ('Men', 'Apparel', 'Topwear'): ['Bottomwear', 'Shoes', 'Accessories'],
    ('Men', 'Apparel', 'Tshirts'): ['Jeans', 'Casual Shoes', 'Watches'],
    ('Men', 'Apparel', 'Shirts'): ['Trousers', 'Formal Shoes', 'Watches'],
    ('Men', 'Apparel', 'Sweatshirts'): ['Jeans', 'Casual Shoes', 'Caps'],
    ('Men', 'Apparel', 'Jackets'): ['Trousers', 'Boots', 'Accessories'],

    # Men's Apparel - Bottomwear
    ('Men', 'Apparel', 'Bottomwear'): ['Topwear', 'Shoes', 'Belts'],
    ('Men', 'Apparel', 'Jeans'): ['Tshirts', 'Sneakers', 'Belts'],
    ('Men', 'Apparel', 'Trousers'): ['Shirts', 'Formal Shoes', 'Belts'],

    # Men's Footwear
    ('Men', 'Footwear', 'Shoes'): ['Apparel', 'Accessories'],
    ('Men', 'Footwear', 'Formal Shoes'): ['Shirts', 'Trousers', 'Watches'],
    ('Men', 'Footwear', 'Casual Shoes'): ['Tshirts', 'Jeans', 'Accessories'],
    ('Men', 'Footwear', 'Sneakers'): ['Tshirts', 'Jeans', 'Caps'],

    # Men's Accessories
    ('Men', 'Accessories', 'Watches'): ['Apparel', 'Footwear'],
    ('Men', 'Accessories', 'Belts'): ['Apparel', 'Footwear'],
    ('Men', 'Accessories', 'Wallets'): ['Apparel', 'Footwear'],
    ('Men', 'Accessories', 'Caps'): ['Tshirts', 'Jeans', 'Sneakers'],
    ('Men', 'Accessories', 'Backpacks'): ['Apparel', 'Footwear'],

    # Women's Apparel - Dresses and Full Outfits
    ('Women', 'Apparel', 'Dress'): ['Accessories', 'Footwear'],
    ('Women', 'Apparel', 'Dresses'): ['Footwear', 'Accessories'],
    ('Women', 'Apparel', 'Lehenga Choli'): ['Jewellery', 'Footwear', 'Accessories'],
    ('Women', 'Apparel', 'Sarees'): ['Jewellery', 'Footwear', 'Accessories'],
    ('Women', 'Apparel', 'Night suits'): ['Loungewear and Nightwear'],
    ('Women', 'Apparel', 'Nightdress'): ['Loungewear and Nightwear'],

    # Women's Apparel - Topwear
    ('Women', 'Apparel', 'Topwear'): ['Bottomwear', 'Shoes', 'Accessories'],
    ('Women', 'Apparel', 'Tshirts'): ['Bottomwear', 'Footwear', 'Accessories'],
    ('Women', 'Apparel', 'Shirts'): ['Bottomwear', 'Footwear', 'Accessories'],
    ('Women', 'Apparel', 'Sweatshirts'): ['Bottomwear', 'Footwear', 'Accessories'],
    ('Women', 'Apparel', 'Kurtas'): ['Leggings', 'Footwear', 'Accessories'],
    ('Women', 'Apparel', 'Tops'): ['Bottomwear', 'Footwear', 'Accessories'],
    ('Women', 'Apparel', 'Jackets'): ['Bottomwear', 'Shoes', 'Accessories'],

    # Women's Apparel - Bottomwear
    ('Women', 'Apparel', 'Bottomwear'): ['Topwear', 'Shoes', 'Accessories'],
    ('Women', 'Apparel', 'Skirts'): ['Tops', 'Accessories', 'Footwear'],
    ('Women', 'Apparel', 'Leggings'): ['Kurtas', 'Footwear', 'Accessories'],
    ('Women', 'Apparel', 'Jeans'): ['Topwear', 'Shoes', 'Accessories'],
    ('Women', 'Apparel', 'Trousers'): ['Topwear', 'Footwear', 'Accessories'],
    ('Women', 'Apparel', 'Shorts'): ['Topwear', 'Shoes', 'Accessories'],
    ('Women', 'Apparel', 'Tights'): ['Topwear', 'Footwear'],
    ('Women', 'Apparel', 'Churidar'): ['Kurtas', 'Footwear', 'Accessories'],

    # Women's Footwear
    ('Women', 'Footwear', 'Shoes'): ['Apparel', 'Accessories'],
    ('Women', 'Footwear', 'Sandal'): ['Apparel', 'Accessories'],
    ('Women', 'Footwear', 'Formal Shoes'): ['Apparel', 'Accessories'],
    ('Women', 'Footwear', 'Flip Flops'): ['Apparel', 'Accessories'],
    ('Women', 'Footwear', 'Flats'): ['Apparel', 'Accessories'],
    ('Women', 'Footwear', 'Heels'): ['Apparel', 'Accessories'],
    ('Women', 'Footwear', 'Sports Shoes'): ['Apparel', 'Accessories'],

    # Women's Accessories
    ('Women', 'Accessories', 'Jewellery'): ['Apparel', 'Footwear'],
    ('Women', 'Accessories', 'Jewellery Set'): ['Apparel', 'Footwear'],
    ('Women', 'Accessories', 'Necklace and Chains'): ['Apparel', 'Footwear'],
    ('Women', 'Accessories', 'Ring'): ['Apparel', 'Footwear'],
    ('Women', 'Accessories', 'Earrings'): ['Apparel', 'Footwear'],
    ('Women', 'Accessories', 'Bangle'): ['Apparel', 'Footwear'],
    ('Women', 'Accessories', 'Bracelet'): ['Apparel', 'Footwear'],
    ('Women', 'Accessories', 'Pendant'): ['Apparel', 'Footwear'],
    ('Women', 'Accessories', 'Clutches'): ['Apparel', 'Footwear'],
    ('Women', 'Accessories', 'Bags'): ['Apparel', 'Footwear'],
    ('Women', 'Accessories', 'Backpacks'): ['Apparel', 'Footwear'],
    ('Women', 'Accessories', 'Suspenders'): ['Apparel', 'Footwear'],
    ('Women', 'Accessories', 'Wallets'): ['Apparel', 'Footwear'],
    ('Women', 'Accessories', 'Caps'): ['Apparel', 'Footwear'],
    ('Women', 'Accessories', 'Sunglasses'): ['Apparel', 'Footwear'],

    # Miscellaneous
    ('Women', 'Apparel', 'Loungewear and Nightwear'): ['Footwear', 'Accessories'],
    ('Women', 'Apparel', 'Rompers'): ['Footwear', 'Accessories'],
    ('Women', 'Apparel', 'Tunics'): ['Footwear', 'Accessories'],
    ('Women', 'Apparel', 'Swimwear'): ['Accessories', 'Footwear'],
    ('Men', 'Apparel', 'Tracksuits'): ['Footwear', 'Accessories'],
    ('Women', 'Apparel', 'Salwar and Dupatta'): ['Footwear', 'Accessories'],
    ('Men', 'Accessories', 'Tablet Sleeve'): ['Apparel', 'Footwear'],
    ('Women', 'Accessories', 'Waistcoat'): ['Apparel', 'Footwear'],
    ('Women', 'Footwear', 'Booties'): ['Apparel', 'Accessories']
}

    recommendation_key = (gender, input_category, input_subcategory)
    categories_to_recommend = recommendation_categories.get(recommendation_key)

    if categories_to_recommend is None:
        print(f"No specific recommendation rules found for input: {recommendation_key}")
        return {}

    recommendations = {}
    for category in categories_to_recommend:
        filtered_items = df_encoded[(df_encoded['gender'] == gender) &
                                    (df_encoded['usage'] == input_item['usage']) &
                                    (df_encoded['masterCategory'] == category) &
                                    (df_encoded.index != input_index)].copy()

        if filtered_items.empty:
            print(f"No items found in the '{category}' category for the given input.")
            continue

        # Calculate complementary color similarity (if applicable)
        if category in ['Apparel', 'Footwear', 'Accessories']: # Assuming color is relevant for these
            filtered_items['color_similarity'] = filtered_items.apply(
                lambda row: calculate_complementary_color_similarity(input_item['baseColour'], row['baseColour']), axis=1)
            sorted_items = filtered_items.sort_values(by=['color_similarity', 'ratings'], ascending=False)
        else:
            # For other categories, sort by ratings only
            sorted_items = filtered_items.sort_values(by=['ratings'], ascending=False)

        recommendations[category] = sorted_items.head(num_recommendations)

    return recommendations

# Assuming you have the calculate_complementary_color_similarity and get_rgb_from_color_name functions defined as before

In [ ]:
def calculate_complementary_color_similarity(color1, color2):
    """Calculates the similarity between two colors based on their complementary colors."""
    try:
        # Convert color names to RGB values (you might need a color dictionary)
        rgb1 = colorsys.rgb_to_hsv(*get_rgb_from_color_name(color1))
        rgb2 = colorsys.rgb_to_hsv(*get_rgb_from_color_name(color2))

        # Calculate complementary color of color1
        complementary_rgb1 = (1 - rgb1[0], 1 - rgb1[1], 1 - rgb1[2])

        # Calculate similarity between complementary_rgb1 and rgb2
        similarity = 1 - (abs(complementary_rgb1[0] - rgb2[0]) +
                          abs(complementary_rgb1[1] - rgb2[1]) +
                          abs(complementary_rgb1[2] - rgb2[2])) / 3

        return similarity

    except (TypeError, ValueError):
        # Handle cases where color conversion fails
        return 0

In [ ]:
# You'll need to define the get_rgb_from_color_name function based on your color data
def get_rgb_from_color_name(color_name):
    # Example: If your data has a color dictionary:
    color_dict = {
    'Navy Blue': (0, 0, 0.5),  # Navy
    'Blue': (0, 0, 1),
    'Silver': (0.75, 0.75, 0.75),  # Light Gray
    'Black': (0, 0, 0),
    'Grey': (0.5, 0.5, 0.5),
    'Green': (0, 1, 0),
    'Purple': (0.5, 0, 0.5),
    'White': (1, 1, 1),
    'Beige': (0.96, 0.96, 0.86),
    'Brown': (0.6, 0.4, 0.2),
    'Bronze': (0.8, 0.5, 0.2),
    'Teal': (0, 0.5, 0.5),
    'Copper': (0.72, 0.45, 0.2),
    'Pink': (1, 0.75, 0.8),
    'Off White': (0.98, 0.98, 0.94),
    'Maroon': (0.5, 0, 0),
    'Red': (1, 0, 0),
    'Khaki': (0.94, 0.9, 0.55),
    'Orange': (1, 0.65, 0),
    'Yellow': (1, 1, 0),
    'Charcoal': (0.2, 0.2, 0.2),
    'Steel': (0.54, 0.54, 0.54),
    'Gold': (1, 0.84, 0),
    'Tan': (0.82, 0.71, 0.55),
    'Multi': (0.5, 0.5, 0.5),
    'Magenta': (1, 0, 1),
    'Lavender': (0.9, 0.9, 0.98),
    'Sea Green': (0.18, 0.55, 0.34),
    'Cream': (1, 0.99, 0.82),
    'Peach': (1, 0.89, 0.71),
    'Olive': (0.5, 0.5, 0),
    'Burgundy': (0.5, 0, 0.13),
    'Coffee Brown': (0.44, 0.33, 0.18),
    'Grey Melange': (0.6, 0.6, 0.6),
    'Rust': (0.8, 0.4, 0.1),
    'Rose': (1, 0.5, 0.5),
    'Lime Green': (0.2, 1, 0),
    'Turquoise Blue': (0.25, 0.88, 0.82),
    'Metallic': (0.75, 0.75, 0.75),
    'Mustard': (1, 0.89, 0.1),
    'Skin': (0.98, 0.8, 0.69),
    'Taupe': (0.28, 0.24, 0.2),
    'Nude': (0.94, 0.88, 0.82),
    'Mushroom Brown': (0.6, 0.56, 0.47),
    'Mauve': (0.86, 0.6, 0.69),  # Light Purple-Pink
    'Fluorescent Green': (0.75, 1, 0.15) # Bright Green
}

    return color_dict.get(color_name, (0, 0, 0))  # Default to black if color not found

In [ ]:
input_index = 309
recommendations = recommend_complementary_items(input_index)
for category, items in recommendations.items():
    print(f"\nRecommended {category}:")
    print(items[['gender', 'articleType', 'baseColour', 'usage', 'ratings', 'Price (USD)', 'link']])




Recommended Apparel:
      gender articleType baseColour   usage  ratings  Price (USD)  \
20172    Men      Shirts    Magenta  Casual        5           37   

                                                    link  
20172  http://assets.myntassets.com/v1/images/style/p...  

Recommended Footwear:
   gender articleType baseColour   usage  ratings  Price (USD)  \
37    Men  Flip Flops       Blue  Casual        5           38   

                                                 link  
37  http://assets.myntassets.com/v1/images/style/p...  


In [ ]:
def print_item_details(index):
    """Prints the details of an item at the given index in df_encoded."""
    try:
        item = df_encoded.iloc[index]
        print(f"Item details at index {index}:")
        for column, value in item.items():
            print(f"{column}: {value}")
    except IndexError:
        print(f"Error: Invalid index {index}. Index out of bounds.")

In [ ]:
print_item_details(303)

Item details at index 303:
gender: Men
masterCategory: Footwear
subCategory: Shoes
articleType: Sports Shoes
baseColour: Black
usage: Sports
ratings: 5
Price (USD): 56
link: http://assets.myntassets.com/v1/images/style/properties/4f04caea8e9056499b866fba1bef20f1_images.jpg
gender_Boys: 0.0
gender_Girls: 0.0
gender_Men: 1.0
gender_Unisex: 0.0
gender_Women: 0.0
baseColour_Beige: 0.0
baseColour_Black: 1.0
baseColour_Blue: 0.0
baseColour_Bronze: 0.0
baseColour_Brown: 0.0
baseColour_Burgundy: 0.0
baseColour_Charcoal: 0.0
baseColour_Coffee Brown: 0.0
baseColour_Copper: 0.0
baseColour_Cream: 0.0
baseColour_Fluorescent Green: 0.0
baseColour_Gold: 0.0
baseColour_Green: 0.0
baseColour_Grey: 0.0
baseColour_Grey Melange: 0.0
baseColour_Khaki: 0.0
baseColour_Lavender: 0.0
baseColour_Lime Green: 0.0
baseColour_Magenta: 0.0
baseColour_Maroon: 0.0
baseColour_Mauve: 0.0
baseColour_Metallic: 0.0
baseColour_Multi: 0.0
baseColour_Mushroom Brown: 0.0
baseColour_Mustard: 0.0
baseColour_Navy Blue: 0.0
baseCo